In [1]:
import numpy as np
import optuna
import torch

from image_classification.models.mlp import MLPClassifier
from image_classification.models.resnet import ResNetClassifier_fastai, Resnet
import lightning.pytorch as pl
import lightning.pytorch.loggers as pl_loggers
import lightning.pytorch.callbacks as pl_callbacks
from image_classification.models.classification_model import ClassificationModel

from util import set_seed
from weight_init import init_for_relu
from sklearn.model_selection import ParameterGrid
from datasets import FashionMNIST
from experiment import train
import sklearn.metrics as skm
import torch
from image_classification.models.cnn import CNN
from image_classification.models.mobilenet import Mobilenet
from weight_init import generic_init
import torchvision.models as models

C:\Users\George Tzoupis\anaconda3\envs\ailab\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def show_classification_results(test_dl, model, classnames):
    ypred, ytrue = [], []

    model.eval()
    with torch.inference_mode():
        for x, y in test_dl:
            pred = model(x.to(model.device))
            ypred.extend(pred.argmax(-1).cpu().numpy().flatten())
            ytrue.extend(y.cpu().numpy().flatten())

    print(skm.classification_report(ytrue, ypred, target_names=classnames))
    print(skm.confusion_matrix(ytrue, ypred))

# Param Grid using sklearn

In [3]:
torch.set_float32_matmul_precision("medium")

In [4]:
param_grid = {
    'n_features': [
        # (4, 8, 16, 32, 64),
        # (8, 16, 32, 64, 128),
        # (16, 32, 64, 128, 256), # L
        (32, 64, 128, 256, 512),  # XL
    ],
    'lr': [
        # 1e-3,
        3e-2,
    ],
    'wd': [
        1e-5,
        # 1e-4,
    ],
    'batch_sz': [32]
}

for params in ParameterGrid(param_grid):
    pl.seed_everything(42, True)

    train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=params['batch_sz'],
                                                               pin_memory=True,
                                                               num_workers=4,
                                                               persistent_workers=True)
    n_classes = len(train_dl.dataset.classes)

    # Select classification model:

    # classifier = MLPClassifier(
    #     input_sz=28 * 28,
    #     n_classes=n_classes,
    #     n_features=(16, 32,),
    # ).apply(init_for_relu)

    # classifier = ResNetClassifier_fastai(
    #     n_classes=n_classes,
    #     n_features=params["n_features"]
    # ).apply(generic_init)

    # classifier = CNN(
    #     n_input_channels=1,
    #     n_classes=n_classes,
    #     n_features=params["n_features"],
    #     n_hidden_layers=128,
    #     init_weights=True,
    #     use_sepconv=False,
    # )

    # classifier = Resnet(
    #     n_input_channels=1,
    #     n_classes=n_classes,
    #     n_features=params["n_features"],
    #     n_hidden_layers=128,
    #     use_fft_input=False,
    #     init_weights=True,
    # )

    classifier = Mobilenet(
        n_input_channels=1,
        n_classes=n_classes,
        n_features=params["n_features"],
        n_hidden_layers=128,
        init_weights=True,
    )

    model = ClassificationModel(
        model=classifier,
        n_classes=n_classes,
        opt="AdamW",
        lr=params["lr"],
        wd=params["wd"]
    )

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    experiment_name = f"Mobilenet3x-{params['n_features'][0]}"
    # experiment_name = f"Testing-Refactor"
    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name=experiment_name,
                                             sub_dir=model.classifier.__class__.__name__)
    trainer = pl.Trainer(max_epochs=20,
                         # limit_train_batches=10,
                         # limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )
    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    show_classification_results(test_dl, model, test_dl.dataset.classes)


Global seed set to 42
C:\Users\George Tzoupis\anaconda3\envs\ailab\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./results\Mobilenet3x-32
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type                | Params
---------------------------------------------------
0 | classifier | Mobilenet           | 253 K 
1 | accuracy   | MulticlassAccuracy  | 0     
2 | precision  | MulticlassPrecision | 0     
3 | recall     | MulticlassRecall    | 0     
---------------------------------------------------
253 K     Trainable

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


              precision    recall  f1-score   support

 T-shirt/top       0.86      0.89      0.88      1000
     Trouser       0.99      0.98      0.99      1000
    Pullover       0.89      0.91      0.90      1000
       Dress       0.92      0.91      0.91      1000
        Coat       0.84      0.92      0.88      1000
      Sandal       0.99      0.95      0.97      1000
       Shirt       0.79      0.68      0.73      1000
     Sneaker       0.92      0.98      0.95      1000
         Bag       0.98      0.99      0.99      1000
  Ankle boot       0.97      0.95      0.96      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

[[894   0  19  15   2   1  63   1   5   0]
 [  0 976   0  16   2   0   2   0   2   2]
 [ 12   2 909   7  41   0  28   0   1   0]
 [  6   1   7 911  20   0  53   0   1   1]
 [  0   1  28  21 918   0  32   0   0   0]
 [  0   0   0   0   0 950 

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    params = {
        "dropout": trial.suggest_float("dropout", 0.1, 0.5, log=True),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["SGD", "RAdam"]),
    }

    data_module = FashionMNIST.FashionMNISTDataModule()
    n_classes = data_module.n_classes
    model = ResNetClassifier(
        n_classes=n_classes,
        **params).apply(init_for_relu)

    trainer = train(model,
                    data_module,
                    monitored_metric='accuracy/val',
                    mode='max',
                    max_epochs=10,
                    limit_train_batches=200)

    return trainer.callback_metrics["accuracy/val"].item()


pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=60 * 60, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
data_module = FashionMNIST.FashionMNISTDataModule()
ckpt_path = "results/FashionMNIST/version_2/checkpoints/last.ckpt"
model = ResNetClassifier.load_from_checkpoint(ckpt_path)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=1,
    monitor="accuracy/val",
    mode="max",
    filename="best-{epoch:02d}",
    save_last=True,
)

tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                         name=data_module.dataset_name,
                                         sub_dir=model.__class__.__name__)

trainer = pl.Trainer(max_epochs=40,
                     callbacks=[
                         checkpoint_callback,
                     ],
                     logger=tb_logger,
                     )

trainer.fit(model,
            data_module,
            ckpt_path=ckpt_path,
            )